# The Three-Stage Quantum Key Distribution Protocol

**Learning Outcomes:**

Upon completion of this lesson:
1. Students will understand the concept of double lock cryptography.
1. Students will know the working of three-stage quantum key distribution protocol.
1. Students will be able to apply the concepts of orthogonality of basis, measurement and indistinguishability of non-orthogonal states.


## 1 Background

So far we’ve seen protocols that use non-orthogonal basis and states to exchange keys. However, there are other ways of achieving the same goal. The three-stage protocol is one of them.

The Three-stage protocol is based on one of the oldest ideas in secure exchange of messages and only assumes the availability of an authenticated channel. 

Consider Alice wants to secretly send a message to Bob. Both Alice and Bob have locks that are unique and only the other person recognizes as belonging to Alice/Bob. Now the secret message can be put into a strong box and sent to Bob using double lock cryptography. The process is straightforward and shown below. 

![nanomod4-fig1.png](nanomod4-fig1.png)

*Note: There was no key exchange necessary in the above protocol.*

## 2 The 3-Stage QKD Protocol

The three stage protocol is a simple, yet powerful, quantum analogy of the double lock cryptography.

Assume Alice wants to send an arbitrary state $\rvert+\rangle$ to Bob.

Both Alice and Bob are connected via an authenticated channel and have secret rotation transformations $U_a$ and $U_b$ such that $U_a U_b$ = $U_b U_a$

$U_\theta$ may be of the form,

$U_\theta =\begin{bmatrix}cos \theta & -sin \theta \\sin\theta& cos\theta& \end{bmatrix}$ 


The protocol proceeds as follows:

1. Alice applies the secret transformation $U_a$ to $\rvert+\rangle$ and then sends Bob $U_a(\rvert+\rangle)$

2. Bob applies $U_b$ and sends to Alice $U_b U_a(\rvert+\rangle)$

3. Alice applies $U_a^\dagger$ and returns $U_b(\rvert+\rangle)$ to Bob.

4. Bob applies $U_b^\dagger$ and retrieves $\rvert+\rangle$. 

State $\rvert+\rangle$ may be our encryption key or data or output of a quantum algorithm.





## 3 The Protocol Flow

![nanomod4-fig2.png](nanomod4-fig2.png)

Under single qubit implementation the protocol provides perfect secrecy. The protocol is also resistant to multi-photon implementations. 


***
***
## 3-Stage QKD Simulator (Interactive)

**Execute the program and follow the prompt below**

In [1]:
import  cmath, numpy, math

"""
Initialize variables taking user input.
"""
print("\n\n-----------------------------------------------------\n\n")
while(True):
    choice = input("Hello, please create a basis; select an option below\n1. Pre-set Basis \n2. Custom Basis\n")
    if (choice == "1"):
        while(True):
            choice2 = input("Choose one of the following basis (in degrees)\n1. {0,90}\n2. {-45,45}\n3. {30,120}")
            if (choice2 == "1"):
                basis = ("0","90")
                break
            if (choice2 == "2"):
                basis = ("-45","45")
                break
            if (choice2 == "3"):
                basis = ("30","120")
                break
        break
    if (choice == "2"):
        while(True):
            #"Enter a measurement basis in the format {x, y}, where x and y are in degrees:"
            inall=input("Enter a measurement basis in the format {x, y}, where x and y are in degrees:\n").strip("{}").replace(" ","")
            in1 = inall.split(",")[0]
            in2 = inall.split(",")[1]
            if(float(in1)+90 == float(in2) or float(in2)+90 == float(in1)):
                basis = (in1,in2)
                break
            else:
                print("Your basis must be 90 degrees apart!")
        break
basis1 = basis[0]
basis2 = basis[1]
basis = (basis1,basis2)
while(True):
    try:
        binary1 = int(input("Input a binary string (ex. 10110011), this will act as the message\n"), 2)
        break
    except ValueError:
        print("Try Again! String must be in binary format (ex. 10110011)")


messagetosend = format(binary1, 'b')

print("\n\n-----------------------------------------------------\n\n")



-----------------------------------------------------


Hello, please create a basis; select an option below
1. Pre-set Basis 
2. Custom Basis
1
Choose one of the following basis (in degrees)
1. {0,90}
2. {-45,45}
3. {30,120}2
Input a binary string (ex. 10110011), this will act as the message
101


-----------------------------------------------------




In [5]:
# function to generate a unitary transformation matrix
def unitary_matrix(theta_deg):
    theta_rad = math.radians(theta_deg)
    unitary_matrix = numpy.array([[math.cos(theta_rad), -math.sin(theta_rad)],
                     [math.sin(theta_rad), math.cos(theta_rad)]])
    return unitary_matrix

arot = float(input("What is Alices secret rotation (in degrees)\n"))%360
brot = float(input("What is Bob's secret rotation (in degrees)\n"))%360

# Generate unitary transformation matrices for Alice and Bob
asecretrot = unitary_matrix(arot)
bsecretrot = unitary_matrix(brot)

receivedmessage = []
"""
This function takes in a bit to send and applies the three step protocol to the bit as it is sent back and forth from a hypothetical Alice to Bob.
"""
bit_zero = numpy.array([numpy.cos(numpy.radians(float(basis1))), numpy.sin(numpy.radians(float(basis1)))])
bit_one = numpy.array([numpy.cos(numpy.radians(float(basis1) + 90)), numpy.sin(numpy.radians(float(basis1) + 90))])

def sendbit(bit):
    print("Sending bit: "+bit)
    tosend = numpy.array([math.cos(math.radians(float(basis1) + (90 * float(bit)))),math.sin(math.radians(float(basis1) + (90 * float(bit))))])
    print("Sending "+ str(tosend.tolist()))
    amessage = aliceone(asecretrot, tosend)
    print ("Transmitting qubit: " + str(amessage.tolist())+ " to Bob")
    abmessage = bobone(bsecretrot, amessage)
    print("Bob Transmitting qubit: "+ str(abmessage.tolist())+" Back to Alice")
    bmessage = alicetwo(asecretrot, abmessage)
    print("Alice Transmitting qubit: "+ str(bmessage.tolist())+" after applying Hermitian transformation")
    message = bobtwo(bsecretrot,bmessage)
    print("Recieved qubit is: "+str(message.tolist()) + "\nTherefore, the received bit after measurement is "+bit)
    given_vector = numpy.array(message)

    # Define tolerance for comparison
    tolerance = 1e-6
    if numpy.allclose(given_vector, bit_zero, atol=tolerance):
        bit = 0
    elif numpy.allclose(given_vector, bit_one, atol=tolerance):
        bit = 1
    else:
        print("Given vector does not match bit_zero or bit_one within tolerance.")

    receivedmessage.append(bit)
    
"""
Alice takes the message qubit and applies her secret rotation.
"""
def aliceone(secret,qubit):
    print("Applying Alice's secret rotation...")
    return  numpy.dot(secret,qubit)

"""
Bob recieves the qubit from Alice and applies his secret rotation
"""
def bobone(secret, qubit):
    print("Applying Bob's secret rotation...")
    return numpy.dot(secret,qubit)
"""
Alice applies the inverse of her secret rotation so only Bobs rotation is on the qubit.
"""
def alicetwo(secret,qubit):
    print("Applying Alice's inverse secret rotation...")
    return  numpy.dot(secret.conj().T,qubit)

"""
Bob applies the inverse of his secret rotation on the qubit.

Now Bob has just the plain qubit. 
"""
def bobtwo(secret,qubit):
    print("Applying Bob's inverse secret rotation...")
    return  numpy.dot(secret.conj().T,qubit)



for digit in messagetosend:
    sendbit(digit)
    
print("Alice's transmitted message was "+ messagetosend)

print("Bob's recieved message was "+ str(receivedmessage))


What is Alices secret rotation (in degrees)
40
What is Bob's secret rotation (in degrees)
200
Sending bit: 1
Sending [0.7071067811865476, 0.7071067811865476]
Applying Alice's secret rotation...
Transmitting qubit: [0.0871557427476582, 0.9961946980917455] to Bob
Applying Bob's secret rotation...
Bob Transmitting qubit: [0.25881904510252063, -0.9659258262890684] Back to Alice
Applying Alice's inverse secret rotation...
Alice Transmitting qubit: [-0.42261826174069955, -0.9063077870366499] after applying Hermitian transformation
Applying Bob's inverse secret rotation...
Recieved qubit is: [0.7071067811865476, 0.7071067811865476]
Therefore, the received bit after measurement is 1
Sending bit: 0
Sending [0.7071067811865476, -0.7071067811865476]
Applying Alice's secret rotation...
Transmitting qubit: [0.9961946980917455, -0.0871557427476582] to Bob
Applying Bob's secret rotation...
Bob Transmitting qubit: [-0.9659258262890684, -0.25881904510252063] Back to Alice
Applying Alice's inverse secre